In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import json

import tensorflow as tf
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score, davies_bouldin_score
from sklearn.cluster import HDBSCAN

In [ ]:
from google.colab import drive
import os

# Mount google drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/Imperial MLDS/DeepTimeSeriesClustering'
os.chdir(path)

In [ ]:
# Import functions from real_data.ipynb
!pip install import-ipynb
!pip install nbimporter

import import_ipynb
import nbimporter

from real_data import get_price, data_transformer

In [ ]:
# Function to get next run id for logging
def get_next_run_id():
  results_path = os.path.join(path, 'Results')
  log_path = os.path.join(results_path, 'run_logs.csv')
  if os.path.exists(log_path):
    logs = pd.read_csv(log_path, encoding='utf-8')
    next_run = logs['Run ID'].max() + 1
    return next_run
  else:
    logs = pd.DataFrame({
    'Run ID': [],
    'Number of Cluster': [],
    'Selected Cluster': [],
    'Selected Tickers': [],
    'Silhouette Score': [],
    'Davies-Bouldin Score': [],
    'Kruskal-Wallis p-value': [],
    'Selected Cluster Mean Momentum': [],
    'Selected Cluster Momentum Std': [],
    'Selected Cluster Out-of-sample Cumulative Return': [],
    'Selected Cluster Out-of-sample Volatility': [],
    'Selected Cluster Out-of-sample Sharpe Ratio': [],
    'Selected Cluster Outperformance': [],
    'Data Start Date': [],
    'Data End Date': [],
    'Test Start Date': [],
    'Test End Date': [],
    'S&P500 Cumulative Return': [],
    'S&P500 Volatility': [],
    'S&P500 Sharpe Ratio': [],
    })
    logs.to_csv(log_path, index=False)
    return 1

# Function to create Results and run folders if they do not exist
def create_run_folder(run_id):
  results_path = os.path.join(path, 'Results')
  curr_run_folder_path = os.path.join(results_path, f"run_{run_id}")
  os.makedirs(curr_run_folder_path, exist_ok=True)
  return curr_run_folder_path

# Save model, training history and parameters
def save_environment(run_id, model, history, config):
  results_path = os.path.join(path, "Results")
  curr_run_path = create_run_folder(run_id)

  model_path = os.path.join(curr_run_path, "model.keras")
  history_path = os.path.join(curr_run_path, "history.json")
  config_path = os.path.join(curr_run_path, "params.json")

  model.save(model_path)

  with open(history_path, "w") as json_file:
    json.dump(history, json_file, indent=4)

  with open(config_path, "w") as json_file:
    config_to_save = dict(filter(lambda x: x[0] not in ['encoder', 'decoder', 'callbacks', 'optimizer'], config.items()))
    json.dump(config_to_save, json_file, indent=4)

# Save the run's statistics to csv
def save_run(run_id,
             num_cluster,
             selected_cluster,
             selected_tickers,
             silhouette,
             dbi,
             kw_pvalue,
             mean_mmt,
             mmt_std,
             out_sample_cum_return,
             out_sample_volatility,
             out_sample_sharpe,
             outperformance,
             data_start,
             data_end,
             test_start,
             test_end,
             snp500_cum_return,
             snp500_volatility,
             snp500_sharpe):
  results_path = os.path.join(path, 'Results')
  log_path = os.path.join(results_path, 'run_logs.csv')
  logs = pd.read_csv(log_path, encoding='utf-8')
  logs.loc[len(logs)] = [
      run_id,
      num_cluster,
      selected_cluster,
      selected_tickers,
      silhouette,
      dbi,
      kw_pvalue,
      mean_mmt,
      mmt_std,
      out_sample_cum_return,
      out_sample_volatility,
      out_sample_sharpe,
      outperformance,
      data_start,
      data_end,
      test_start,
      test_end,
      snp500_cum_return,
      snp500_volatility,
      snp500_sharpe
  ]
  logs.to_csv(log_path, index=False)

In [ ]:
# Plot loss
def plot_loss(run_id, histories, loss_name, filename="loss.png", combined_losses=None):
    colors = plt.rcParams["axes.prop_cycle"]()
    if combined_losses is None:
        combined_losses = ["loss"]
    other_losses = [l for l in loss_name if l not in combined_losses]
    num_charts = len(other_losses)
    rows = max(1, (num_charts + 1) // 2)
    fig = plt.figure(figsize=(15, 15))

    # Combined losses subplot (larger, top)
    ax_total = plt.subplot2grid((rows + 1, 2), (0, 0), colspan=2)
    for i, history in enumerate(histories):
        c = next(colors)["color"]  # Same color for all losses in one history
        for loss in combined_losses:
          title = loss.title()
          ylabel = 'Loss'
          if title == 'Loss':
            title = 'Total'
          elif title == 'Learning_Rate':
            title = 'Learning Rate'
            ylabel = 'lr'
          ax_total.plot(history.history[loss], color=c, label=f"Run {i+1}" if len(histories) > 1 else f"{title}")
    ax_total.set_title("Total Loss")
    ax_total.set_xlabel("Epochs")
    ax_total.set_ylabel(ylabel)
    if len(histories) > 1 or len(combined_losses) > 1:
        ax_total.legend()

    # Grid for other losses
    if num_charts > 0:
        axes = [plt.subplot2grid((rows + 1, 2), (i // 2 + 1, i % 2)) for i in range(num_charts)]
        for i, loss in enumerate(other_losses):
          title = loss.title()
          ylabel = 'Loss'
          if title == 'Loss':
            title = 'Total'
          elif title == 'Learning_Rate':
            title = 'Learning Rate'
            ylabel = 'lr'
          for j, history in enumerate(histories):
              c = next(colors)["color"]  # Same color for all losses in one history
              axes[i].plot(history.history[loss], color=c, label=f"Run {j+1}" if len(histories) > 1 else f"{title}")
          axes[i].set_title(f"{title}")
          axes[i].set_xlabel("Epochs")
          axes[i].set_ylabel(ylabel)
          if len(histories) > 1:
              axes[i].legend()

    plt.tight_layout()

    results_path = os.path.join(path, "Results")
    curr_run_path = create_run_folder(run_id)
    img_path = os.path.join(curr_run_path, filename)
    plt.savefig(img_path)


    plt.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from matplotlib.lines import Line2D

# Plot PCA and t-SNE
def plot_pca_tsne(run_id, encoded, labels, filename="pca_tsne.png"):
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
  noise_mask = labels == -1

  unique_clusters = np.unique(labels[~noise_mask])
  palette = sns.color_palette('deep', len(unique_clusters))
  cluster_colors = dict(zip(unique_clusters, palette))

  # PCA
  pca = PCA(n_components=2)
  z_pca = pca.fit_transform(encoded)
  noise_scatter = ax1.scatter(z_pca[noise_mask, 0], z_pca[noise_mask, 1], c='grey', alpha=0.3, label='Noise')
  sns.scatterplot(x=z_pca[~noise_mask, 0], y=z_pca[~noise_mask, 1], hue=labels[~noise_mask], palette=cluster_colors, alpha=1, ax=ax1, legend=False)
  ax1.set_title('PCA')

  # t-SNE
  tsne = TSNE(n_components=2)
  z_tsne = tsne.fit_transform(encoded)
  ax2.scatter(z_tsne[noise_mask, 0], z_tsne[noise_mask, 1], c='grey', alpha=0.3, label='Noise')
  sns.scatterplot(x=z_tsne[~noise_mask, 0], y=z_tsne[~noise_mask, 1], hue=labels[~noise_mask], palette=cluster_colors, alpha=1, ax=ax2, legend=False)
  ax2.set_title('t-SNE')

  handles = [noise_scatter]
  labels_legend = ['Noise']
  for cluster in unique_clusters:
    handles.append(Line2D([0], [0], marker='o', color='w', markerfacecolor=cluster_colors[cluster], markersize=8))
    labels_legend.append(str(cluster))

  fig.legend(handles, labels_legend, loc='center right', bbox_to_anchor=(1.1, 0.5), title='Clusters')

  plt.tight_layout()

  results_path = os.path.join(path, "Results")
  curr_run_path = create_run_folder(run_id)
  img_path = os.path.join(curr_run_path, filename)
  plt.savefig(img_path, bbox_inches='tight')

  plt.show()

In [ ]:
# Functions to evaluate synthetic data on multiple runs
def synthetic_data_evaluation(run_id, num_runs, params, model, synthetic_dataset, synthetic_data_tensor, ground_truth_labels):
  synthetic_historical_data = []
  predicted_labels_list = []
  encoded_list = []
  ari_scores = []
  nmi_scores = []
  silhouette_scores = []
  db_scores = []

  for run in range(num_runs):
    print(f"Running Run {run+1:}")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    synthetic_data_ae_clustering = model(
                dataset=synthetic_dataset,
                encoder=params["encoder"],
                decoder=params["decoder"],
                latent_dim=params["latent_dim"],
                pretrain_epochs=params["pretrain_epochs"],
                min_samples=params["min_samples"],
                max_clusters=params["max_clusters"],
                gamma=params["gamma"],
                alpha=params["alpha"],
                verbose=0
    )
    synthetic_data_ae_clustering.compile(optimizer=optimizer)
    synthetic_history = synthetic_data_ae_clustering.fit(synthetic_dataset, epochs=params["epochs"], batch_size=params["batch_size"], verbose=0, callbacks=[params["callbacks"]])
    synthetic_historical_data.append(synthetic_history)

    # Encode synthetic data and predict labels
    encoded = synthetic_data_ae_clustering.encoder(synthetic_data_tensor)[0]
    z_np_normalized = encoded / (np.std(encoded, axis=0, keepdims=True) + 1e-6) # Normalize latent representations for stability
    encoded = z_np_normalized
    clusterer = HDBSCAN(min_cluster_size=params['min_samples'], min_samples=params['min_samples'], cluster_selection_method='eom')
    predicted_labels = clusterer.fit_predict(encoded)
    predicted_labels_list.append(predicted_labels)

    # Compute ARI and NMI
    ari_score = adjusted_rand_score(ground_truth_labels, predicted_labels)
    nmi_score = normalized_mutual_info_score(ground_truth_labels, predicted_labels)

    valid_indices = predicted_labels != -1
    if sum(valid_indices) > 1:  # If more than 1 valid clusters (exclude noise)
        silhouette = silhouette_score(encoded[valid_indices], predicted_labels[valid_indices])
        db = davies_bouldin_score(encoded[valid_indices], predicted_labels[valid_indices])
    else:
        silhouette = np.nan
        db = np.nan

    ari_scores.append(ari_score)
    nmi_scores.append(nmi_score)
    silhouette_scores.append(silhouette)
    db_scores.append(db)
    print(f"Run {run+1} results: ARI = {ari_score:.4f}, NMI = {nmi_score:.4f}, Silhouette Score = {silhouette:.4f}, DBI = {db:.4f}\n")

  mean_ari = np.mean(ari_scores)
  std_ari = np.std(ari_scores)

  mean_nmi = np.mean(nmi_scores)
  std_nmi = np.std(nmi_scores)
  print(f"Mean ARI: {mean_ari:.4f} +/- {std_ari:.4f}")
  print(f"Mean NMI: {mean_nmi:.4f} +/- {std_nmi:.4f}")


  mean_silhouette = np.nanmean(silhouette_scores)
  std_silhouette = np.nanstd(silhouette_scores)

  mean_db = np.nanmean(db_scores)
  std_db = np.nanstd(db_scores)
  print(f"Mean Silhouette: {mean_silhouette:.4f} +/- {std_silhouette:.4f}")
  print(f"Mean DB: {mean_db:.4f} +/- {std_db:.4f}")

  encoded_list.append(encoded)

  return ari_scores, nmi_scores, silhouette_scores, db_scores, encoded_list, synthetic_historical_data, predicted_labels_list

In [ ]:
# Compute silhouette score and Davies-Bouldin index
def external_clustering_metrics(encoded, metadata, labels):
  non_noise_z_np = tf.gather(encoded, indices=metadata[metadata['Cluster']!=-1].index)
  silh_score = silhouette_score(non_noise_z_np, labels[labels!=-1])
  print(f"Silhouette Score: {silh_score:.4f}")
  db_score = davies_bouldin_score(non_noise_z_np, labels[labels!=-1])
  print(f"Davies-Bouldin Score: {db_score:.4f}")
  return silh_score, db_score

In [ ]:
from scipy.stats import kruskal

# Compute kruskal-wallis p-value to evaluate clusters significance
def momentum_distribution(run_id, metadata, labels):
  cluster_momentum = metadata.groupby('Cluster')['Momentum'].agg(['mean', 'std', 'count']).reset_index()
  print("\nMomentum Statistics by Cluster:")
  display(cluster_momentum)

  # Krusal-wallis p-value calculation for non noise clusters
  if len(np.unique(labels[labels != -1])) > 1:
    momentum_by_cluster = [metadata[metadata['Cluster'] == c]['Momentum'].values
                          for c in np.unique(labels[labels != -1])]
    stat, p_value = kruskal(*momentum_by_cluster)
    print(f"Kruskal-Wallis p-value: {p_value:.4f}")


  plt.figure(figsize=(10, 6))
  for cluster in np.unique(labels[labels != -1]):
      cluster_data_mmt = metadata[metadata['Cluster'] == cluster]['Momentum']
      plt.hist(cluster_data_mmt, bins=20, alpha=0.5, label=f'Cluster {cluster}')
  plt.title('Momentum Distribution by Cluster')
  plt.xlabel('Momentum Score')
  plt.ylabel('Frequency')
  plt.legend()

  results_path = os.path.join(path, "Results")
  curr_run_path = create_run_folder(run_id)
  img_path = os.path.join(curr_run_path, "momentum_distribution.png")
  plt.savefig(img_path)

  plt.show()

  return p_value

In [ ]:
# Compute normalized momentum scores over lookback periods
def compute_momentum_scores(data, lookbacks=[5, 10, 20]):
  returns = data[:, :, 0].numpy()
  volume = data[:, :, 1].numpy()
  momentum_scores = []
  # Aggregate momentum for each lookback period
  for lookback in lookbacks:
    volume_mean = np.mean(volume[:, -lookback:], axis=1, keepdims=True) + 1e-4
    volume_weights = volume[:, -lookback:] / volume_mean
    cum_returns = np.sum(returns[:, -lookback:] * volume_weights, axis=1)
    cum_returns = (cum_returns - np.mean(cum_returns)) / (np.std(cum_returns) + 1e-4)
    momentum_scores.append(cum_returns)
  return np.mean(momentum_scores, axis=0)

# Compute normalized momentum scores over lookback periods for a dataframe
def compute_momentum_scores_df(df, lookbacks=[5, 10, 20]):
  momentum_scores = []
  for ticker in df['Ticker'].unique():
    stock_data = df[df['Ticker'] == ticker].sort_values('Date')
    returns = stock_data['r_close'].values
    volume = stock_data['norm_log_volume'].values
    ticker_scores = []
    # Aggregate momentum for each lookback period
    for lookback in lookbacks:
        if len(returns) >= lookback:
            volume_mean = np.mean(volume[-lookback:]) + 1e-6
            volume_weights = volume[-lookback:] / volume_mean
            cum_return = np.sum(returns[-lookback:] * volume_weights)
            ticker_scores.append(cum_return)
        else:
            print(f"Warning: {ticker} has {len(returns)} time steps, insufficient for lookback {lookback}")
            ticker_scores.append(0.0)
    momentum_scores.append(np.mean(ticker_scores) if ticker_scores else 0.0)
  momentum_scores = np.array(momentum_scores)
  momentum_scores = (momentum_scores - np.mean(momentum_scores)) / (np.std(momentum_scores) + 1e-6)
  return momentum_scores

In [ ]:
# Compute cluster metrics
def compute_cluster_metrics(df, out_sample_start, out_sample_end, tickers):
  df_copy = df.copy()
  df_copy = df_copy[(df_copy['Date'] >= out_sample_start) & (df_copy['Date'] <= out_sample_end)]
  idx = df_copy['Ticker'].isin(tickers)
  cluster_tickers = df_copy[idx]['Ticker'].unique()
  if len(cluster_tickers) == 0:
      return {
          'Cluster': -1,
          'Cumulative Return': 0.0,
          'Volatility': 0.0,
          'Sharpe Ratio': 0.0,
          'Momentum': 0.0,
          'Tickers': []
      }

  cluster_df = df_copy[idx]
  daily_returns = cluster_df[cluster_df['Date'] > out_sample_start].groupby('Date')['r_close'].mean()
  cumulative_return = (1 + daily_returns).prod() - 1 # Cumulative returns
  volatility = daily_returns.std() * np.sqrt(252) # Annualized volatility
  sharpe_ratio = cumulative_return / (volatility + 1e-4) if volatility > 0 else 0.0 # Simplified sharpe ratio (without risk free rate)
  momentum = np.mean(compute_momentum_scores_df(cluster_df)) if len(cluster_tickers) > 0 else 0.0 # Average cluster's normalized momentum

  return {
        'Cumulative Return': cumulative_return,
        'Volatility': volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Momentum': momentum,
        'Tickers': cluster_tickers.tolist()
    }

In [ ]:
# In-sample evaluation
def in_sample_evaluation(start_date, end_date, metadata):
  cluster_momentum = metadata.groupby('Cluster')['Momentum'].mean().reset_index()
  valid_clusters = cluster_momentum[cluster_momentum['Cluster'] != -1]
  num_clusters = len(valid_clusters)
  if len(valid_clusters) == 0:
      raise ValueError("No valid clusters found. Adjust HDBSCAN parameters.")

  # Identify high-momentum cluster
  high_momentum_cluster = valid_clusters.loc[valid_clusters['Momentum'].idxmax(), 'Cluster']
  high_momentum_tickers = metadata[metadata['Cluster'] == high_momentum_cluster]['Ticker'].unique()

  # Display high-momentum cluster's statistics
  print(f"High-momentum cluster: {high_momentum_cluster}")
  print(f"Stocks in high-momentum cluster: {high_momentum_tickers}")
  print(f"\nIn-Sample Momentum Statistics ({start_date} to {end_date}):")
  display(cluster_momentum[cluster_momentum['Cluster'] != -1][['Cluster', 'Momentum']])
  print(f"\nIn-Sample Momentum Statistics For Noise Group ({start_date} to {end_date}):")
  display(cluster_momentum[cluster_momentum['Cluster'] == -1][['Cluster', 'Momentum']])

  high_momentum_cluster_stats = metadata[metadata['Cluster'] == high_momentum_cluster].groupby('Cluster')['Momentum'].agg(['mean', 'std', 'count']).reset_index()

  return num_clusters, high_momentum_cluster, high_momentum_tickers, high_momentum_cluster_stats['mean'].values[0], high_momentum_cluster_stats['std'].values[0]

In [ ]:
# Out-of-sample evaluation
def out_sample_evaluation(run_id,
                          in_sample_df,
                          out_sample_df,
                          data_start,
                          strat_start,
                          out_sample_start,
                          out_sample_end,
                          metadata,
                          high_momentum_cluster):
  combined_df = pd.concat([
      in_sample_df[in_sample_df['Date'] >= data_start],
      out_sample_df
  ]).reset_index(drop=True)

  combined_df['Date'] = pd.to_datetime(combined_df['Date'])
  combined_df = combined_df.sort_values(["Ticker", "Date"], ascending=[True, True])

  combined_df_transformed = data_transformer(combined_df)
  combined_df_transformed_copy = combined_df_transformed.copy()
  combined_df_transformed_copy = combined_df_transformed_copy[combined_df_transformed_copy['Date'] >= strat_start]

  cluster_metrics = []
  cluster_momentum = metadata.groupby('Cluster')['Momentum'].mean().reset_index()
  valid_clusters = cluster_momentum[cluster_momentum['Cluster'] != -1]  # Exclude noise
  # Compute statistics and metrics for each cluster
  for cluster in valid_clusters['Cluster']:
      cluster_tickers = metadata[metadata['Cluster'] == cluster]['Ticker'].unique()
      metrics = compute_cluster_metrics(combined_df_transformed_copy, out_sample_start, out_sample_end, cluster_tickers)
      metrics['Cluster'] = cluster
      cluster_metrics.append(metrics)
  cluster_metrics = pd.DataFrame(cluster_metrics)

  # Retrive S&P500 from the same out-of-sample period for comparison
  snp500_out_sample_filename = f'snp500_out_sample_{out_sample_start.replace("-", "")}_{out_sample_end.replace("-", "")}.csv'
  snp500_out_sample_path = os.path.join(path, snp500_out_sample_filename)
  if not os.path.isfile(snp500_out_sample_path):
    snp500_df = get_price("SPY", out_sample_start, out_sample_end)
    snp500_df['r_close'] = snp500_df['Close'].pct_change().apply(lambda x: np.log(1 + x))
    snp500_df.to_csv(snp500_out_sample_filename)
  else:
    snp500_df = pd.read_csv(snp500_out_sample_path, encoding='utf-8')

  snp500_daily_returns = snp500_df['r_close']
  snp500_cum_return = (1 + snp500_daily_returns).prod() - 1 # Cumulative returns
  snp500_volatility = snp500_daily_returns.std() * np.sqrt(252) # Annualized volatility
  snp500_sharpe = snp500_cum_return / (snp500_volatility + 1e-4) if snp500_volatility > 0 else 0.0 # Simplified sharpe ratio (without risk free rate)

  # If more than 10 non-noise clusters, display all. Otherwise, display only the top 10 in terms of cumulative return
  if len(valid_clusters) <= 10:
    print(f"\nOut-of-Sample Cluster Performance ({out_sample_start} - {out_sample_end}):")
    display(cluster_metrics[['Cluster', 'Cumulative Return', 'Volatility', 'Sharpe Ratio', 'Momentum', 'Tickers']])
  else:
    print(f"\nTop 10 & High-Momentum Cluster Out-of-Sample Cluster Performance ({out_sample_start} - {out_sample_end}):")
    cluster_metrics_top_10 = cluster_metrics.nlargest(10, 'Cumulative Return')
    cluster_metrics_selected = cluster_metrics[cluster_metrics['Cluster'] == high_momentum_cluster]
    if high_momentum_cluster not in cluster_metrics_top_10['Cluster'].unique():
      cluster_metrics = pd.concat([cluster_metrics_selected, cluster_metrics_top_10]).sort_values('Cumulative Return', ascending=False)
    cluster_metrics.reset_index(inplace=True, drop=True)
    display(cluster_metrics[['Cluster', 'Cumulative Return', 'Volatility', 'Sharpe Ratio', 'Momentum', 'Tickers']])


  cluster_cum_return = cluster_metrics[cluster_metrics['Cluster'] == high_momentum_cluster]['Cumulative Return'].values[0]
  cluster_volatility = cluster_metrics[cluster_metrics['Cluster'] == high_momentum_cluster]['Volatility'].values[0]
  cluster_sharpe = cluster_metrics[cluster_metrics['Cluster'] == high_momentum_cluster]['Sharpe Ratio'].iloc[0]
  outperformance = cluster_cum_return - snp500_cum_return

  print(f"\nHigh-Momentum Cluster (Cluster {high_momentum_cluster}) Outperformance:")
  print(f"\tCumulative Return vs. S&P 500: {outperformance:.4f}")

  print(f"\nHigh-Momentum Cluster Performance ({out_sample_start} - {out_sample_end}):")
  print(f"\tCumulative Return: {cluster_cum_return:.4f}")
  print(f"\tVolatility: {cluster_volatility:.4f}")
  print(f"\tSharpe Ratio: {cluster_sharpe:.4f}")

  print(f"\nS&P 500 Performance ({out_sample_start} - {out_sample_end}):")
  print(f"\tCumulative Return: {snp500_cum_return:.4f}")
  print(f"\tVolatility: {snp500_volatility:.4f}")
  print(f"\tSharpe Ratio: {snp500_sharpe:.4f}")

  print("\nMomentum Comparison:")
  momentum_comparison = cluster_momentum[cluster_momentum['Cluster'] != -1][['Cluster', 'Momentum']].rename(columns={'Momentum': 'In-Sample Momentum'})
  momentum_comparison = momentum_comparison.merge(
      cluster_metrics[['Cluster', 'Momentum']].rename(columns={'Momentum': 'Out-of-Sample Momentum'}),
      on='Cluster'
  )
  display(momentum_comparison)


  # Visualize returns
  plt.figure(figsize=(10, 6))

  cluster_metrics['Selected'] = False
  cluster_metrics.loc[cluster_metrics['Cluster']==high_momentum_cluster, 'Selected'] = True

  color_palette = {True: 'skyblue', False: 'gray'}

  sns.barplot(cluster_metrics, x='Cluster', y='Cumulative Return', hue='Selected', palette=color_palette, dodge=False)
  plt.axhline(snp500_cum_return, color='orange', linestyle='--')
  plt.text(len(cluster_metrics)-1, snp500_cum_return, 'S&P 500', va='bottom', color='orange')
  plt.title(f'Out-of-Sample Cumulative Returns ({out_sample_start} - {out_sample_end})')
  plt.xlabel('Cluster')
  plt.ylabel('Cumulative Return')
  plt.legend(title='Selected Cluster')

  results_path = os.path.join(path, "Results")
  curr_run_path = create_run_folder(run_id)
  img_path = os.path.join(curr_run_path, "outperformance.png")
  plt.savefig(img_path)

  plt.show()


  # Visualize momentum
  plt.figure(figsize=(10, 6))
  plt.plot(momentum_comparison['Cluster'], momentum_comparison['In-Sample Momentum'], marker='o', label='In-Sample Momentum')
  plt.plot(momentum_comparison['Cluster'], momentum_comparison['Out-of-Sample Momentum'], marker='s', label='Out-of-Sample Momentum')
  plt.title('Momentum Comparison: In-Sample vs. Out-of-Sample')
  plt.xlabel('Cluster')
  plt.ylabel('Momentum Score')
  plt.legend()
  plt.show()

  return cluster_cum_return, cluster_volatility, cluster_sharpe, snp500_cum_return, snp500_volatility, snp500_sharpe, outperformance